In [ ]:
## Can also get the price chart by pokemon number, or url for past prices. Look intro trading strategies; also, work on real-time offering feed vs history. 

In [2]:
import time
import os
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# Dynamically set CSV file path to your desktop
CSV_FILE = os.path.join(os.path.expanduser("~"), "Desktop", "price_data.csv")


# Global dataframe variable to store scraped data
price_df = pd.DataFrame()


def scrape_price_and_listings():
    """
    Opens the TCGplayer page for Gardevoir-ex,
    waits for price and listings elements, and extracts their text.
    """
    driver = webdriver.Chrome()
    try:
        url = "https://www.tcgplayer.com/product/518689/pokemon-prize-pack-series-cards-gardevoir-ex?Condition=Near+Mint&Language=English&page=1"
        driver.get(url)

        wait = WebDriverWait(driver, 20)

        price_elem = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "spotlight__price"))
        )
        price_text = price_elem.text.strip()

        listings_elem = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "filter-bar"))
        )
        listings_text = listings_elem.text.strip()

        print(f"Price: {price_text}, Listings: {listings_text}")
        return price_text, listings_text

    finally:
        driver.quit()


def append_to_csv_and_df(price_text, listings_text):
    """
    Converts the scraped price and listings text into numeric types,
    calculates price*listing as an integer,
    appends new data row with timestamp to both a global dataframe and the CSV file on desktop.
    Timestamp column is saved and used as dataframe index.
    """
    global price_df

    # Validate input presence
    if not price_text or not listings_text:
        print("Warning: Empty price or listings text; skipping this scrape.")
        return

    timestamp = datetime.now()

    # Convert price string (e.g. "$4.99") to float safely
    try:
        price_num = float(price_text.replace('$', '').replace(',', ''))
    except ValueError:
        print(f"Warning: Could not convert price '{price_text}' to float; skipping.")
        return

    # Extract digits and convert listings string (e.g. "16 Listings") to int safely
    listings_digits = ''.join(filter(str.isdigit, listings_text))
    if not listings_digits:
        print(f"Warning: Could not find numeric value in listings '{listings_text}'; skipping.")
        return

    listings_num = int(listings_digits)

    # Calculate price*listing and convert to int
    price_times_listings = int(price_num * listings_num)

    # Prepare new row as dataframe
    new_data = {
        "timestamp": [timestamp],
        "price": [price_num],          # keep as float
        "listings": [listings_num],   # int
        "price*listing": [price_times_listings]
    }
    df_new = pd.DataFrame(new_data)

    try:
        # Read existing CSV with timestamp parsing
        df_existing = pd.read_csv(CSV_FILE, parse_dates=['timestamp'])
        price_df = pd.concat([df_existing, df_new], ignore_index=True)
    except FileNotFoundError:
        price_df = df_new

    # Set datetime as index
    price_df.set_index('timestamp', inplace=True)

    # Save the combined dataframe to CSV
    price_df.to_csv(CSV_FILE)

    print(f"Appended new data to {CSV_FILE}")


def main():
    while True:
        price_text, listings_text = scrape_price_and_listings()
        append_to_csv_and_df(price_text, listings_text)
        print("Waiting for 1 hour before next scrape...\n")
        time.sleep(3600)  # wait for 1 hour


if __name__ == "__main__":
    main()


Price: $4.98, Listings: 17 Listings
Appended new data to /Users/nshaffer/Desktop/price_data.csv
Waiting for 1 hour before next scrape...



KeyboardInterrupt: 

In [ ]:
df_new

NameError: name 'df_new' is not defined